In [3]:
from anet_api.db import Team, Athlete, Meet
import requests as r
from sqlmodel import select, Session
from anet_api.db.database import SessionLocal
import json

In [4]:
api_url = 'http://127.0.0.1:8000/'

In [5]:
def get_all_teams(session: Session):
    stmt = select(Team)
    return session().exec(stmt).all()

def insert_meets_and_athletes(params):
    q = r.get(api_url + 'team/getInfo', params = params)
    meets = q.json()['schedule']
    roster = q.json()['roster']
    for meet in meets:
        response = r.post(api_url + 'meet/addMeet',data=json.dumps(meet))
        if response.status_code != 200:
            print(meet['name'], response.text)
    for athlete in roster:
        response = r.post(api_url + 'athlete/addAthlete', data=json.dumps(athlete))
        if response.status_code != 200:
            print(athlete['anet_id'], response.text)
    return

In [6]:
season = 2023
sport = 'xc'
for team in get_all_teams(SessionLocal):
    print(team)
    #params = dict(team_id = team.anet_id, season = season, sport = sport)
    #insert_meets_and_athletes(params)

In [5]:
from sqlalchemy import extract
def get_all_meets_by_season(session: Session, season: int):
    stmt = select(Meet).where(extract('year', Meet.date) == season)
    return session.exec(stmt).all()

In [10]:

meets = get_all_meets_by_season(SessionLocal(),2023)




In [11]:
meets

[]

In [8]:
def insert_new_result(meet_id: int):
    params = dict(sport = 'xc', meet_id = meet_id)
    s = r.get(api_url + 'meet/getResults',params=params)
    s.json()

    all_results = []
    for race in s.json()['races']:
        all_results += race['results']

    for result in all_results:
        response = r.post(api_url + 'meet/addResult', data = json.dumps(result))
        if response.status_code == 422:
            print(response.text)
    return "All done!"

In [9]:
for meet in meets:
    insert_new_result(meet.anet_id)

In [26]:
session = SessionLocal()
team = Team(anet_id=111, name="New Team")
athlete = Athlete(anet_id=222, first_name="John", last_name="Doe", gender="F")
meet = Meet(anet_id=333, name="New Meet", venue="Place", date="2023-01-01")

session.add(team)
session.add(athlete)
session.add(meet)
session.commit()

In [13]:
import requests as r
url = 'https://www.athletic.net/api/v1/Event/Events'
data = dict(
        start='2022-11-01',
        end='2022-11-30',
        levelMask=4,
        sportMask=2,
        state='WA',
        country='us',
    )

res = r.post(url,json=data)

In [14]:
res.json()

{'events': [{'IDMeet': 224294,
   'Sport': 'XC',
   'MeetName': 'Forks HS League Meet (CANCELED)',
   'SeasonId': 2023,
   'StartDate': '0001-01-01T00:00:00',
   'EndDate': '2023-09-19T00:00:00',
   'Virtual': False,
   'LocationName': 'Olympic Natural Resources Center',
   'StreetAddress': '1455 S Forks Ave',
   'City': 'Forks',
   'PostalCode': '98331',
   'State': 'WA',
   'OwnerID': 364,
   'CalendarLock': 0,
   'UCalendarLock': 0,
   'RegEnd': None,
   'HasResults': 0,
   'LevelMask': 6,
   'HostName': 'Forks',
   'MascotUrl': '//lh3.googleusercontent.com/AUH2hZZ7KFJd1HnvbCxwUWO5zaQuN2YUpzeckUG9V9bd6Y6BpCO4ajPePXdkY0zjbVNaEfaKkhQsHPFr08_kMw',
   'Lat': 47.9330028,
   'Long': -124.3944981,
   'score': 0.0,
   'CalCount': 1,
   'OffDays': '{}',
   'Data': None,
   'rsUrl': None},
  {'IDMeet': 225599,
   'Sport': 'XC',
   'MeetName': 'Removed',
   'SeasonId': 2023,
   'StartDate': '0001-01-01T00:00:00',
   'EndDate': '2023-10-14T00:00:00',
   'Virtual': False,
   'LocationName': 'Oce